In [0]:
import numpy as np
import random
from IPython.display import clear_output

from statistics import mean
from random import randrange
from copy import deepcopy

In [157]:
Q = np.array([[0,0,0,0,0,0,0,0,0]])
S = np.array([[0,0,0,0,0,0,0,0,0]])
actual_state_index = 0
state_index_1 = 0
state_index_2 = 0 
state_index_3 = 0
action = 0
action_1 = 0
action_2 = 0
action_3 = 0
print_all = False
print_q = False
actual_state_vector = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0])
my_path = "/content/drive/My Drive/Colab Notebooks/Q-Learning/q-learning_titactoe/"
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'ignore', 'invalid': 'ignore', 'over': 'warn', 'under': 'ignore'}

In [0]:
Win = np.array([0,0,0])

gamma = 0.8
epsilon = 0.2

In [0]:
def clear_game():
  global actual_state_index
  global state_index_1
  global state_index_2
  global state_index_3
  global action
  global actual_state_vector

  actual_state_index = 0
  state_index_1 = 0
  state_index_2 = 0
  state_index_3 = 0
  action = 0
  actual_state_vector = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0])

def add_state(new_state, player_symbol):
  global S
  global Q
  global actual_state_index
  global state_index_1
  global state_index_2
  global state_index_3
  global actual_state_vector

  state_index_3 = state_index_2
  state_index_2 = state_index_1
  state_index_1 = actual_state_index

  index = np.where((S[:,0] == new_state[0]) 
                  & (S[:,1] == new_state[1]) 
                  & (S[:,2] == new_state[2]) 
                  & (S[:,3] == new_state[3]) 
                  & (S[:,4] == new_state[4]) 
                  & (S[:,5] == new_state[5]) 
                  & (S[:,6] == new_state[6]) 
                  & (S[:,7] == new_state[7])
                  & (S[:,8] == new_state[8]))

  if (len(index[0]) == 0):
    actual_state_index = len(S)
    actual_state_vector = new_state
    S = np.vstack([S, new_state])
    Q = np.vstack([Q, [0,0,0,0,0,0,0,0,0]])
  else:
    actual_state_index = index[0][0]
    actual_state_vector = S[index[0][0]].copy()

def map(value):
  if (value == 1):
     return "o"
  elif (value == 2):
    return "x"
  else:
    return " "

def drow_actual_state():
  print(map(actual_state_vector[0]) + "|" + map(actual_state_vector[1]) + "|" + map(actual_state_vector[2]) +
      "\n" + map(actual_state_vector[3]) + "|" + map(actual_state_vector[4]) + "|" + map(actual_state_vector[5]) +
      "\n" + map(actual_state_vector[6]) + "|" + map(actual_state_vector[7]) + "|" + map(actual_state_vector[8]) + "\n")

def get_vector_index(row, col):
  return (row-1)*3+col-1

def make_move_row_col(player_symbol, row, col):
  action = get_vector_index(row,col)
  return make_move(player_symbol)

def make_move(player_symbol):
  global action_3
  global action_2
  global action_1

  if (actual_state_vector[action] == 0):
    action_3 = action_2
    action_2 = action_1
    action_1 = action
    actual_state_vector[action] = player_symbol
    add_state(actual_state_vector, player_symbol)
    return True
  else:
    print("Invalid action!")
    return False

def random_move(player_symbol):
  global action

  avalible_actions = [i for i, j in enumerate(actual_state_vector) if j == 0]
  if (avalible_actions != []):
    action = random.choice(avalible_actions)
    make_move(player_symbol) 

def switch_player(player_symbol):
  if (player_symbol == 1):
    return 2
  else:
    return 1  

def check_win(player_symbol):
  global action
  global actual_state_vector

  best_action = -1
  avalible_actions = [i for i, j in enumerate(actual_state_vector) if j == 0]
  vector_to_return = actual_state_vector.copy()

  win = False
  for a in avalible_actions:
    action = a
    actual_state_vector[a] = player_symbol
    if (check_win_player(player_symbol) == 1):
      win = True
      best_action = action
    actual_state_vector = vector_to_return.copy()

  if (win == True):
    action = best_action  

  return win  

def check_win_lose(player_symbol):
  win = check_win(player_symbol)
  if (win == True):
    make_move(player_symbol)
    add_reward(player_symbol)
    return True

  lose = check_win(switch_player(player_symbol))
  if (lose == True):
    make_move(player_symbol)
    add_reward(player_symbol)
    return True

  return False  

def q_move(player_symbol, learn):
  global action

  avalible_actions = [i for i, j in enumerate(actual_state_vector) if j == 0]

  if (avalible_actions != []):
    action = np.where(Q[actual_state_index] == np.amax(Q[actual_state_index,avalible_actions]))[0]

    if (len(action) > 1):
      action = random.choice(list(set(avalible_actions).intersection(action)))
    
    if (str(type(action)) == "<class 'numpy.ndarray'>"): #czasami action zamiast być liczbą jest jedno elementową tablicą
      action = action[0]

    make_move(player_symbol)
    add_reward(player_symbol)

def add_reward(player_symbol):
    avalible_actions = [i for i, j in enumerate(S[state_index_3]) if j == 0]
    if (avalible_actions != []):
      reward = Q[state_index_3, action_3] + gamma * mean(Q[state_index_1,avalible_actions]) #funkcja nagrody
      Q[state_index_3, action_3] = reward #przypisanie nagrody do Q

def set_reward_if_win(player_symbol):
  win = check_win_player(player_symbol)
  if (win == 1):
    Q[state_index_1][action] = 100
    Q[state_index_2][action_2] = -100
    Win[player_symbol] = Win[player_symbol] + 1
    if (print_all == True):
      print('player ' + str(player_symbol) + ' won!')
    return True
  elif (win == 2):
    Q[state_index_1][action] = 10
    Q[state_index_2][action_2] = 10
    Win[0] = Win[0] + 1
    if (print_all == True):
      print('Draw!')
    return True
  else:
    return False  

def check_win_player(player):
  if (((actual_state_vector[0] == player) & (actual_state_vector[1] == player) & (actual_state_vector[2] == player)) |
      ((actual_state_vector[3] == player) & (actual_state_vector[4] == player) & (actual_state_vector[5] == player)) |
      ((actual_state_vector[6] == player) & (actual_state_vector[7] == player) & (actual_state_vector[8] == player)) |
      ((actual_state_vector[0] == player) & (actual_state_vector[3] == player) & (actual_state_vector[6] == player)) |
      ((actual_state_vector[1] == player) & (actual_state_vector[4] == player) & (actual_state_vector[7] == player)) |
      ((actual_state_vector[2] == player) & (actual_state_vector[5] == player) & (actual_state_vector[8] == player)) |
      ((actual_state_vector[0] == player) & (actual_state_vector[4] == player) & (actual_state_vector[8] == player)) |
      ((actual_state_vector[2] == player) & (actual_state_vector[4] == player) & (actual_state_vector[6] == player))):
    return 1
  elif (len(actual_state_vector[actual_state_vector == 0]) == 0):
    return 2
  else:
    return 3      

def player_move(player_symbol):
  global action

  valid_move = False
  while (valid_move == False):
    action = int(input())-1;
    valid_move = make_move(player_symbol)

  add_reward(player_symbol)  

  print('\n')
  drow_actual_state()
  return set_reward_if_win(player_symbol) 

def random_computer_move(player_symbol, learn):
  random_move(player_symbol)
  if (print_all == True):
    drow_actual_state()
  return set_reward_if_win(player_symbol)      

def q_computer_move(player_symbol, learn):
  if (check_win_lose(player_symbol) == False):
    if (random.uniform(0, 1) < epsilon):
      random_move(player_symbol)
    else:  
      q_move(player_symbol, learn)

  if (print_all == True):
    drow_actual_state()
  return set_reward_if_win(player_symbol) 

def play_game(learn = False, learn_2 = False):
  global Q

  clear_game()
  if (print_all == True):
    drow_actual_state()
  i = 0
  win = False

  while win==False:
    if (print_q == True):
      print(Q[actual_state_index])
    i = i + 1
    if (i%2==1):
      if (learn == True):
        win = q_computer_move(1, learn)
      else:  
        win = player_move(1)
    else:  
      if (learn_2 == True):
        win = q_computer_move(2, learn)
      else:
        win = player_move(2)

  if (np.amax(Q) >= 100):
    Q = Q / np.amax(Q) * 100

def play_with_computer():
  global print_all

  print_all = True
  play_game(False, True)

def learn():
  global print_all

  print_all = False
  for i in range(100000):
    if (i%10000==0):
      print(Win)
      print(Win / sum(Win))
      print(i)
    play_game(True, True)

  print(Win)
  print(Win / sum(Win))
  print("finish")  

def Save(name = ""):
  np.savetxt(my_path + "Q_" + name + ".csv", Q, delimiter=",")
  np.savetxt(my_path + "S_" + name + ".csv", S, delimiter=",")

def Load(name = ""):
  global Q
  global S

  Q = np.genfromtxt(my_path + "Q_" + name + ".csv", delimiter=',')
  S = np.genfromtxt(my_path + "S_" + name + ".csv", delimiter=',')

In [160]:
learn()

[0 0 0]
[nan nan nan]
0
[8444 1013  543]
[0.8444 0.1013 0.0543]
10000
[17150  1911   939]
[0.8575  0.09555 0.04695]
20000
[25892  2774  1334]
[0.86306667 0.09246667 0.04446667]
30000
[34878  3499  1623]
[0.87195  0.087475 0.040575]
40000
[44064  4102  1834]
[0.88128 0.08204 0.03668]
50000
[53251  4679  2070]
[0.88751667 0.07798333 0.0345    ]
60000
[62396  5263  2341]
[0.89137143 0.07518571 0.03344286]
70000
[71746  5680  2574]
[0.896825 0.071    0.032175]
80000
[80966  6234  2800]
[0.89962222 0.06926667 0.03111111]
90000
[90323  6675  3002]
[0.90323 0.06675 0.03002]
finish


In [161]:
play_with_computer()

 | | 
 | | 
 | | 

5


 | | 
 |o| 
 | | 

 | | 
 |o| 
 | |x

3


 | |o
 |o| 
 | |x

 | |o
 |o| 
x| |x

8


 | |o
 |o| 
x|o|x

 |x|o
 |o| 
x|o|x

1


o|x|o
 |o| 
x|o|x

o|x|o
 |o|x
x|o|x

4


o|x|o
o|o|x
x|o|x

Draw!
